# Using MVR to predict Multi-XRF from well log data

Put together by Thomas Martin, thomasmartin@mines.edu, all errors are mine

In [ ]:
!pip install scikit-learn --upgrade

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.24.1)


In [ ]:
# If you have installation questions, please reach out
import pandas as pd # data storage
import numpy as np  # math and stuff

import datetime

import sklearn  
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, median_absolute_error

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt # plotting utility

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/Merged_dataset_inner_imputed_12_21_2020.csv')

df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num', 'sample' ], axis=1)

df.describe()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,depth_ft,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,PE_smooth,GR_smooth,gz_linear_interp,gz_pchip_interp,perm_air_md,perm_klink_md,He_por,horz_perm_md,effective_por,por_percent,stress_swanson_perm_md,Na,Na Err,Mg,Mg Err,Al,Al Err,Si,Si Err,P,P Err,S,S Err,K,K Err,Ca,Ca Err,Ti,Ti Err,Cr,Cr Err,Mn,Mn Err,Fe,Fe Err,Co,Co Err,Ni,Ni Err,Cu,Cu Err,Zn,Zn Err,Ba,Ba Err
count,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,508.000000,510.000000,510.000000,25.000000,25.000000,24.000000,24.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.0,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000
mean,5397.863358,0.468406,0.325999,0.430844,0.811259,0.613371,0.351642,1.011018,0.172737,0.172737,0.351702,0.326029,4.913576,4.910244,81.002187,73.797216,14.267039,8.304000,11.808000,8.420833,1.646585,1.132042,0.025387,0.762913,0.011739,3.490170,0.012272,32.981494,0.033185,0.048516,0.001821,0.728955,0.002247,1.675288,0.005710,2.779623,0.005543,0.190546,0.001373,0.005787,0.000116,0.016520,0.000705,1.941109,0.003196,0.001315,0.0,0.004773,0.000100,0.003211,0.000822,0.006309,0.000763,0.090289,0.004689
std,5106.156677,0.191316,0.237404,0.263514,0.162843,0.214604,0.186912,1.503608,0.226664,0.226664,0.178735,0.229901,1.738128,1.751499,173.917355,161.251984,9.365678,3.161814,4.446902,3.050728,3.741652,0.777851,0.004405,0.488501,0.002843,1.415185,0.002607,4.940991,0.002003,0.023636,0.000221,0.795110,0.001252,0.761818,0.001383,2.045850,0.002189,0.088087,0.000297,0.002270,0.000075,0.024982,0.000128,1.472163,0.001193,0.000821,0.0,0.000796,0.000002,0.004429,0.000114,0.004559,0.000100,0.065308,0.000594
min,150.000000,0.000000,0.000000,0.003752,0.000000,0.000000,0.000000,-0.744314,0.000570,0.000570,0.004477,0.000258,1.204296,1.199972,0.001000,0.000000,1.000000,1.900000,5.900000,4.000000,0.000390,0.126200,0.016000,0.047000,0.006800,0.992200,0.007100,8.429500,0.021700,0.000000,0.001100,0.040300,0.000600,0.118400,0.002000,0.133500,0.001400,0.048500,0.000800,0.000000,-0.001200,0.000000,0.000500,0.270700,0.001300,0.000200,0.0,0.002700,0.000100,0.000000,0.000200,0.000000,0.000500,0.000000,0.003100
25%,450.725000,0.360072,0.122318,0.174146,0.743878,0.424460,0.216157,-0.037336,0.004595,0.004595,0.220800,0.131058,3.538724,3.531571,0.040750,0.020250,8.300000,7.900000,8.400000,6.800000,0.001140,0.768500,0.023400,0.293850,0.009100,2.411025,0.010200,29.728050,0.032000,0.032400,0.001700,0.182225,0.001200,1.069725,0.004600,1.103025,0.003600,0.121875,0.001100,0.004400,0.000100,0.003200,0.000600,0.964250,0.002300,0.000700,0.0,0.004300,0.000100,0.000000,0.000700,0.002700,0.000700,0.053200,0.004300
50%,1614.650000,0.554827,0.275825,0.471657,0.867010,0.605516,0.329327,0.795008,0.005532,0.005532,0.337049,0.283192,4.307541,4.256191,0.065500,0.037500,9.300000,9.500000,10.600000,7.300000,0.002070,1.057300,0.025200,0.692000,0.011700,3.133250,0.011700,31.893100,0.033300,0.047250,0.001800,0.461450,0.001900,1.560450,0.005800,2.593250,0.005700,0.160300,0.001300,0.005400,0.000100,0.014200,0.000700,1.658800,0.003100,0.001200,0.0,0.004800,0.000100,0.000000,0.000800,0.005400,0.000800,0.079300,0.004700
75%,9649.575000,0.602566,0.530980,0.662807,0.925012,0.830179,0.470706,1.378387,0.332503,0.332503,0.463317,0.525448,6.615175,6.633493,71.375000,59.675000,26.610000,10.300000,13.800000,9.200000,0.949777,1.322750,0.026700,1.192400,0.014400,4.531475,0.014400,37.944075,0.034700,0.061800,0.002000,0.941100,0.002900,2.291400,0.006900,3.872700,0.007000,0.258025,0.001600,0.006800,0.000100,0.023400,0.000800,2.616775,0.003900,0.001700,0.0,0.005200,0.000100,0.006300,0.000900,0.009200,0.0

## Loading in Dataset

In [ ]:
df = df[df.Al >= 0] # making the dataset only XRF data

df.columns.values

array(['depth_ft', 'USGS_ID', 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 'PE_smooth', 'GR_smooth',
       'gz_linear_interp', 'gz_pchip_interp', 'perm_air_md',
       'perm_klink_md', 'He_por', 'horz_perm_md', 'effective_por',
       'por_percent', 'stress_swanson_perm_md', 'Na', 'Na Err', 'Mg',
       'Mg Err', 'Al', 'Al Err', 'Si', 'Si Err', 'P', 'P Err', 'S',
       'S Err', 'K', 'K Err', 'Ca', 'Ca Err', 'Ti', 'Ti Err', 'Cr',
       'Cr Err', 'Mn', 'Mn Err', 'Fe', 'Fe Err', 'Co', 'Co Err', 'Ni',
       'Ni Err', 'Cu', 'Cu Err', 'Zn', 'Zn Err', 'Ba', 'Ba Err'],
      dtype=object)

In [ ]:
dataset = df[[
  'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
  'RESD', 'PHIN', 'PHID', 'GR_smooth', 'PE_smooth', 
  'Al', 'Ti', 'Si', 'Mg', 'Ca'
]]

In the next code block, we will remove the rows without data, and change string NaN's to np.nans

In [ ]:
dataset.replace('NaN',np.nan, regex=True, inplace=True)# 

np.shape(dataset)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


(2396, 16)

In [ ]:
dataset.head(3)

,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,GR_smooth,PE_smooth,Al,Ti,Si,Mg,Ca
0,0.630495,0.187403,0.054025,0.582081,0.829176,0.349137,0.982069,0.001692,0.001692,0.183350,0.366294,2.8101,0.1641,33.9921,0.6987,4.5640
5,0.626112,0.182393,0.063209,0.586778,0.825487,0.351442,0.922670,0.001810,0.001810,0.196369,0.357995,2.9962,0.1746,33.9915,0.9547,3.8975
10,0.621729,0.184446,0.063426,0.590484,0.825878,0.402086,0.886733,0.001889,0.001889,0.211611,0.360458,2.5388,0.1555,33.3257,0.7553,4.1047


In [ ]:
X = dataset[[ 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 'GR_smooth', 'PE_smooth']]

Y = dataset[['Al', 'Ti', 'Si', 'Mg', 'Ca']]


## Starting to set up the ML model params

In [ ]:
seed = 7 # random seed is only used if you want to compare exact answers with friends 
test_size = 0.25 # how much data you want to withold, .15 - 0.3 is a good starting point

X_train, X_test, y_train, y_test = train_test_split(X.values, Y.values, test_size=test_size)

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

LinearRegression()

In [ ]:
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

Intercept: 
 [ 2.05675826  0.21168139 26.07192549  1.33855429  7.23795775]
Coefficients: 
 [[ 4.67041564e-01 -4.01754661e+00 -3.95141260e-01 -6.41851341e-01
   1.06988536e+00 -7.96099092e-01 -1.07364687e-01  5.26913684e-01
   5.26913684e-01  7.19932578e+00  1.20770034e+00]
 [-1.72786781e-02 -2.53389189e-01 -4.63725492e-02 -4.48470874e-02
  -3.97068402e-02 -7.26460817e-02 -5.71385003e-03 -1.46219598e-02
  -1.46219598e-02  4.60516695e-01  1.03505157e-01]
 [ 2.21080486e+00  8.58974453e+00  1.02226248e+00 -1.42764722e+00
   1.03299655e+01  1.00316373e+00  2.88937872e-01  5.34198282e+00
   5.34198282e+00 -1.65447549e+01  9.64728388e-01]
 [-2.42996969e-01 -1.71077657e+00 -8.12343499e-02 -4.04913852e-01
  -4.53903634e-01 -1.97368520e-01 -3.32646518e-03 -5.48401963e-01
  -5.48401963e-01  2.88340185e+00  1.61321068e-01]
 [-1.84108004e+00  5.66537365e-01 -4.59316109e-01  2.11609365e-01
  -4.00175043e+00  4.68746475e-01  3.18626790e-02 -2.82814321e+00
  -2.82814321e+00  8.40550048e-01 -2.29401850

In [ ]:
preds = regr.predict(X_test)

In [ ]:
rmse5 = mean_squared_error(y_test, preds, squared=False)
print("Mean Squared Error: %f" % (rmse5))
#max5 = max_error(y_test, preds)
#print("Max Error: %f" % (max5))

MAE2 = median_absolute_error(y_test, preds)
print("Median Abs Error: %f" % (MAE2))

Mean Squared Error: 1.341304
Median Abs Error: 0.713931


In [ ]:
x = datetime.datetime.now()

d = {'target': ['MultiXRF'],
     'MSE': [rmse5 ],
     'MAE': [MAE2],
     'day': [x.day], 
     'month':[x.month], 
     'year':[x.year],
     'model':['MVR'],
     'version':[sklearn.__version__]}

results = pd.DataFrame(data=d)

results.to_csv('drive/My Drive/1_lewis_research/analysis/experiments/mvr/mvr_results/multi_XRF_mvr.csv')

results

,target,MSE,MAE,day,month,year,model,version
0,MultiXRF,1.341304,0.713931,15,2,2021,MVR,0.24.1


In [ ]:
preds

array([[ 3.48502288,  0.19604329, 34.37543472,  0.9326812 ,  2.9425698 ],
       [ 3.35620966,  0.18874371, 32.54529151,  0.76706844,  2.52617942],
       [ 3.76119178,  0.21770553, 29.61812635,  1.03231555,  3.64441009],
       ...,
       [ 2.39396711,  0.15093061, 30.41219865,  0.75795441,  4.13489972],
       [ 2.53513037,  0.10671083, 41.17945421,  0.21637373,  0.9358294 ],
       [ 2.8329354 ,  0.17997411, 28.7741802 ,  0.99452113,  4.56682894]])

In [ ]:
y_test

array([[ 2.9122,  0.157 , 35.8347,  0.6212,  3.4171],
       [ 1.898 ,  0.119 , 30.759 ,  0.3818,  5.5049],
       [ 2.1348,  0.1192, 32.4912,  0.5667,  2.8806],
       ...,
       [ 1.9976,  0.1263, 27.2278,  0.5551,  7.2091],
       [ 3.0767,  0.1274, 40.1429,  0.1545,  0.3163],
       [ 2.85  ,  0.1739, 22.749 ,  1.225 ,  7.3786]])